In [1]:
import os
from keras.layers import Input, Dense, Flatten, Reshape
from sklearn.feature_extraction import image as sk_image
from concurrent.futures import ProcessPoolExecutor
from keras.models import Model
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from sklearn.cluster import AgglomerativeClustering
from scipy.signal import convolve2d
from sklearn.cluster import KMeans
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from pyhdf.SD import SD, SDC
import matplotlib as mpl
#tf.config.threading.set_inter_op_parallelism_threads(1)
from extract_training_data import *
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
from pyhdf.error import HDF4Error
from functions import *
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping



In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)

#bands = [6, 7, 20, 28, 28, 31]


1 Physical GPUs, 1 Logical GPUs


2023-09-26 16:24:16.259040: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21034 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:25:00.0, compute capability: 7.5


In [3]:
#bands = [6,20,29]
bands = [1]


In [4]:
import autoencoder
import importlib
importlib.reload(autoencoder)
from autoencoder import SobelFilterLayer, SimpleAutoencoder
patch_size = 64

autoencoder = SimpleAutoencoder(len(bands), patch_size, patch_size)
#x = autoencoder.normalize(x)
#optimizer = mixed_precision.LossScaleOptimizer(tf.keras.optimizers.Adam(learning_rate=1e-4), loss_scale='dynamic')
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model = autoencoder.model(optimizer=optimizer, loss="combined")


Input should already be normalized. Call self.normalize to normalize list of data


In [5]:
val_data = np.float32(np.load("/scratch/fslippe/modis/MOD02/test_data/normalized_testpatches_band(1)_winter20_21.npy"))
patches = np.float32(np.load("/scratch/fslippe/modis/MOD02/training_data/normalized_trainingpatches_band(1)_winter20_21.npy"))
print(patches.shape)
print(val_data.shape)



(300169, 64, 64, 1)
(33353, 64, 64, 1)


In [6]:
### STANDARD FIT
gc.collect()
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

model.fit(patches, patches, validation_data=(val_data, val_data), epochs=200, batch_size=32, callbacks=[early_stopping])


2023-09-26 16:24:23.193275: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4917968896 exceeds 10% of free system memory.
2023-09-26 16:24:24.457542: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4917968896 exceeds 10% of free system memory.
2023-09-26 16:24:26.914194: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4917968896 exceeds 10% of free system memory.
2023-09-26 16:24:28.685988: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4917968896 exceeds 10% of free system memory.


Epoch 1/200


2023-09-26 16:24:32.539096: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202


9381/9381 [==============================] - 75s 8ms/step - loss: 0.0072 - val_loss: 0.0051
Epoch 2/200
1041/9381 [==>...........................] - ETA: 1:00 - loss: 0.0050

KeyboardInterrupt: 

In [ ]:
model.save("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(1)_autoencoder")
autoencoder.encoder.save("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(1)_encoder")
autoencoder.decoder.save("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(1)_decoder")


2023-09-26 16:05:54.645588: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(1)_autoencoder/assets


/opt/software/easybuild/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/software/easybuild/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


INFO:tensorflow:Assets written to: /uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(1)_encoder/assets


/opt/software/easybuild/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/software/easybuild/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


INFO:tensorflow:Assets written to: /uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(1)_decoder/assets
